In [3]:
import numpy as np
import torch

from math import ceil
import matplotlib.pyplot as plt

from neuralhedge.nn import datahedger, contigent, mlp, loss
from neuralhedge.market import stochastic
from neuralhedge._utils.plotting import plot_pnl, plot_history

from importlib import reload
reload(stochastic)
reload(loss)
reload(contigent)
reload(datahedger)
torch.manual_seed(1)


In [4]:
n_paths = 5000
step_size = 1/365
maturity = 30/365
n_steps = ceil(maturity / step_size)
initial_value = 1.
strike = initial_value
mu = 0.0
sigma = 0.2

blackscholes = stochastic.BlackScholes(mu = mu, 
                            sigma = sigma,
                            n_paths = n_paths,
                            n_steps = n_steps,
                            step_size = step_size)
blackscholes.stimulate(initial_value = initial_value)
option = contigent.EuropeanVanilla(strike = strike)

paths = blackscholes.prices     # (n_paths, n_steps+1, n_asset)
information = torch.cat([
    torch.log(blackscholes.prices),
    blackscholes.times_inverse
    ], axis = -1)
payoff = option.payoff(blackscholes.prices)   # (n_paths, n_steps+1, 1)

n_asset = paths.shape[-1]
data_set = [paths, information, payoff]
